# Observable Trends

###### 1)
###### 2)
###### 3)

In [14]:
# Import dependencies
import pandas as pd

In [15]:
# Set csv paths
schools_path = 'raw_data/schools_complete.csv'
students_path = 'raw_data/students_complete.csv'

In [16]:
# Read schools csv
schools_df = pd.read_csv(schools_path)
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [17]:
# Read students csv
students_df = pd.read_csv(students_path)
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary

In [18]:
# Grab total schools
total_schools = schools_df['name'].count()

# Grab total students
total_students = schools_df['size'].sum()

# Grab total budget
total_budget = schools_df['budget'].sum()

# Grab average math score
avg_math = students_df['math_score'].mean()

# Grab average reading score
avg_read = students_df['reading_score'].mean()

# Calculate % passing math
pass_math = (students_df['math_score'] >= 70).sum()/total_students*100

# Calculate % passing reading
pass_read = (students_df['reading_score'] >= 70).sum()/total_students*100

# Calculate overall passing grade
overall_pass = (pass_math + pass_read)/2

In [19]:
# Create District Summary dataframe
district_summary = pd.DataFrame({'Total Schools': [total_schools], 
                                 'Total Students': [total_students], 
                                 'Total Budget': [f'${total_budget:,.2f}'], 
                                 'Average Math Score': [avg_math], 
                                 'Average Reading Score': [avg_read], 
                                 '% Passing Math': [pass_math], 
                                 '% Passing Reading': [pass_read], 
                                 '% Overall Passing Rate': [overall_pass]})

# Reorder columns
district_summary = district_summary[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 
                                     'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [24]:
# Use students dataframe to groupby school
grouped_shools = students_df.groupby(['school'])
grouped_shools

In [26]:
# Grab average math and reading scores for each school
avg_math_scores = grouped_shools['math_score'].mean()
avg_read_scores = grouped_shools['reading_score'].mean()

# Create dataframe from averages
student_scores = pd.DataFrame({'Average Math Score': avg_math_scores, 
                                'Average Reading Score': avg_read_scores})

# Reset index for merge
student_scores = student_scores.reset_index()

# Drop index column
#student_scores = student_scores.drop(['index'], axis=1)
student_scores.head()

,school,Average Math Score,Average Reading Score
0,Bailey High School,77.048432,81.033963
1,Cabrera High School,83.061895,83.975780
2,Figueroa High School,76.711767,81.158020
3,Ford High School,77.102592,80.746258
4,Griffin High School,83.351499,83.816757


In [27]:
# Subset original students df to count passing students for each school
only_pass_math = students_df.loc[students_df['math_score'] >= 70,:]
only_pass_read = students_df.loc[students_df['reading_score'] >= 70,:]

# Group by school
grouped_only_pass_math = only_pass_math.groupby(['school'])
grouped_only_pass_read = only_pass_read.groupby(['school'])

# Grab student counts who passed for each school
pass_math = grouped_only_pass_math['math_score'].count()
pass_read = grouped_only_pass_read['reading_score'].count()

In [28]:
# Insert group by series into dataframe
pass_summary = pd.DataFrame({'Passing Math Counts': pass_math, 'Passing Reading Counts': pass_read})

# Reset index
pass_summary = pass_summary.reset_index()
pass_summary.head()

,school,Passing Math Counts,Passing Reading Counts
0,Bailey High School,3318,4077
1,Cabrera High School,1749,1803
2,Figueroa High School,1946,2381
3,Ford High School,1871,2172
4,Griffin High School,1371,1426


In [29]:
# Merge student_scores df and pass_summary on school
merge_stu_pass = pd.merge(student_scores, pass_summary, on='school')
merge_stu_pass.head()

,school,Average Math Score,Average Reading Score,Passing Math Counts,Passing Reading Counts
0,Bailey High School,77.048432,81.033963,3318,4077
1,Cabrera High School,83.061895,83.975780,1749,1803
2,Figueroa High School,76.711767,81.158020,1946,2381
3,Ford High School,77.102592,80.746258,1871,2172
4,Griffin High School,83.351499,83.816757,1371,1426


In [30]:
# Rename columns and set df to schools summary
schools_summary = schools_df.rename(columns={'name': 'school', 'type': 'School Type', 'size': 'Total Students', 
                                             'budget': 'Total School Budget'})
# Drop School ID
schools_summary = schools_summary.drop(['School ID'], axis=1)

# Add budget per student column
schools_summary['Per Student Budget'] = schools_summary['Total School Budget']/schools_summary['Total Students']
schools_summary.head()

,school,School Type,Total Students,Total School Budget,Per Student Budget
0,Huang High School,District,2917,1910635,655.0
1,Figueroa High School,District,2949,1884411,639.0
2,Shelton High School,Charter,1761,1056600,600.0
3,Hernandez High School,District,4635,3022020,652.0
4,Griffin High School,Charter,1468,917500,625.0


In [31]:
# Merge schools_summary and students_summary on school
combined_stu_sch = pd.merge(schools_summary, merge_stu_pass, on='school')

# Add % passing math and reading
combined_stu_sch['% Passing Math'] = combined_stu_sch['Passing Math Counts']/combined_stu_sch['Total Students']*100
combined_stu_sch['% Passing Reading'] = combined_stu_sch['Passing Reading Counts']/combined_stu_sch['Total Students']*100

# Add Overall Passing Rate Column
combined_stu_sch['% Overall Passing Rate'] = (combined_stu_sch['% Passing Math'] + combined_stu_sch['% Passing Reading'])/2

# Drop passing math and reading counts columns
combined_stu_sch = combined_stu_sch.drop(['Passing Math Counts', 'Passing Reading Counts'], axis=1)

# Map to format budget columns
combined_stu_sch['Total School Budget'] = combined_stu_sch['Total School Budget'].map('${:,.2f}'.format)
combined_stu_sch['Per Student Budget'] = combined_stu_sch['Per Student Budget'].map('${:,.2f}'.format)

# Rename school column
combined_stu_sch = combined_stu_sch.rename(columns={'school': 'School Name'})

# Sort alphabetically
combined_stu_sch = combined_stu_sch.sort_values(by=['School Name'])

# Index school for visibility
combined_stu_sch = combined_stu_sch.set_index(['School Name'])
combined_stu_sch

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


# Top Performing Schools (By Passing Rate)

In [32]:
# Sort
top_schools = combined_stu_sch.sort_values(by=['% Overall Passing Rate'], ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Passing Rate)

In [33]:
# Sort
top_schools = combined_stu_sch.sort_values(by=['% Overall Passing Rate'])
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


# Math Scores by Grade

In [34]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [35]:
# Group by grade
grouped_sch_grade = students_df.groupby(['school', 'grade'])
grade_math_score = grouped_sch_grade['math_score'].mean()

# Reset index
grade_math_score = grade_math_score.reset_index()
grade_math_score

# Create dataframe
# grade_math_score = pd.DataFrame(grade_math_score)
# grade_math_score
# test = test.stack(level=['9th', '10th', '11th', '12th'])

,school,grade,math_score
0,Bailey High School,10th,76.996772
1,Bailey High School,11th,77.515588
2,Bailey High School,12th,76.492218
3,Bailey High School,9th,77.083676
4,Cabrera High School,10th,83.154506
5,Cabrera High School,11th,82.765560
6,Cabrera High School,12th,83.277487
7,Cabrera High School,9th,83.094697
8,Figueroa High School,10th,76.539974
9,Figueroa High School,11th,76.884344
